Imports

In [1]:
%matplotlib notebook
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = 9.5, 6
import matplotlib.pyplot as plt

import keras as K
import numpy as np
import tensorflow as tf
import sklearn as skl
import sklearn.model_selection as skl_model_selection
import itertools
import time
from pathlib import Path
import shutil

Using TensorFlow backend.


Obtain and Preprocess Data

In [2]:
%%time
TRUTH_CLASSES = 10
INPUT_DIM = 48

(input_train_orig, truth_train_orig), (input_test_orig, truth_test_orig) = K.datasets.cifar10.load_data()

input_train_big_tensor = tf.image.resize_images(input_train_orig, [INPUT_DIM, INPUT_DIM])
input_test_big_tensor  = tf.image.resize_images(input_test_orig, [INPUT_DIM, INPUT_DIM])

with K.backend.get_session().as_default():
    input_train_big = input_train_big_tensor.eval().astype('uint8')
    input_test_big  = input_test_big_tensor.eval().astype('uint8')
    
input_train_preprocessed = K.applications.vgg19.preprocess_input(input_train_big)
input_test_preprocessed = K.applications.vgg19.preprocess_input(input_test_big)
truth_train = K.utils.to_categorical(truth_train_orig, TRUTH_CLASSES)
truth_test = K.utils.to_categorical(truth_test_orig, TRUTH_CLASSES)

CPU times: user 8.66 s, sys: 3.74 s, total: 12.4 s
Wall time: 12.4 s


Create Callbacks

In [3]:
def get_model_name(num):
    return "model-{num}".format(num=num)

def get_tb_callback(name):
    path = "./exp-graphs/" + name
    return K.callbacks.TensorBoard(log_dir=path, write_graph=False)

def get_checkpointing_callback(name):
    path = "./exp-models/" + name + "-epoch-{epoch:02d}.hdf5"
    return K.callbacks.ModelCheckpoint(filepath=path, verbose=1, save_best_only=True)

Encapsulation of training

In [4]:
OVERTRAINING_FACTOR = 2.5
def create_model(model_no, initial_epochs=6, batch=32):
    K.backend.clear_session()
    np.random.seed(model_no)
    
    name = get_model_name(model_no)
    best_weights = Path("./exp-models") / (name + "-overtrained.hdf5")
    overtrained_weights = Path("./exp-models") / (name + "-overtrained.hdf5")
    graphs_dir = Path("./exp-graph") / name
    
    if not overtrained_weights.exists():
        print(">>> Begin train for model", model_no)
        bag_indexes = np.random.randint(0, len(input_train_preprocessed)-1, len(input_train_preprocessed))
        x_train_bag = input_train_preprocessed[bag_indexes]
        y_train_bag = truth_train[bag_indexes]
    
    
        if best_weights.exists():
            best_weights.unlink()
        if graphs_dir.exists():
            shutil.rmtree(graphs_dir)

        tensorboard_callback = get_tb_callback(name)
        checkpoint_callback = get_checkpointing_callback(name)

        vgg19 = K.applications.vgg19.VGG19(
            include_top=False,
            weights='imagenet',
            input_shape=(INPUT_DIM, INPUT_DIM, 3),
        )
        
        for layer in vgg19.layers:
            layer.trainable = False

        x = K.layers.Flatten()(vgg19.output)
        x = K.layers.Dense(261, activation='relu')(x)
        output = K.layers.Dense(TRUTH_CLASSES, activation='softmax')(x)
        model = K.models.Model(vgg19.input, output)

        model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

        h = model.fit(
            x_train_bag,
            y_train_bag,
            epochs=initial_epochs,
            batch_size=batch,
            shuffle=True,
            verbose=2,
            validation_split=0.2,
            callbacks=[tensorboard_callback, checkpoint_callback]
        )
        best_epoch = np.argmin(h.history["val_loss"]) + 1
        overtrained_epoch = int(best_epoch * OVERTRAINING_FACTOR)
        print(">>> Best epoch found:", best_epoch, "Overtraining to:", overtrained_epoch)
        
        h = model.fit(
            x_train_bag,
            y_train_bag,
            initial_epoch=initial_epochs,
            epochs=overtrained_epoch,
            batch_size=batch,
            shuffle=True,
            verbose=1,
            validation_split=0.2,
            callbacks=[tensorboard_callback]
        )
        model.save(overtrained_weights)

    print(">>> Obtained best and overtrained for model", model_no)
    return best_weights, overtrained_weights

Train all the networks

In [5]:
NUM_MODELS = 100
Path("./exp-models").mkdir(exist_ok=True)
for model_no in range(1, NUM_MODELS+1):
    best_weights_path, overtrained_weights_path = create_model(model_no)

>>> Obtained best and overtrained for model 1
>>> Obtained best and overtrained for model 2
>>> Obtained best and overtrained for model 3
>>> Obtained best and overtrained for model 4
>>> Obtained best and overtrained for model 5
>>> Obtained best and overtrained for model 6
>>> Obtained best and overtrained for model 7
>>> Obtained best and overtrained for model 8
>>> Obtained best and overtrained for model 9
>>> Obtained best and overtrained for model 10
>>> Obtained best and overtrained for model 11
>>> Obtained best and overtrained for model 12
>>> Obtained best and overtrained for model 13
>>> Obtained best and overtrained for model 14
>>> Obtained best and overtrained for model 15
>>> Obtained best and overtrained for model 16
>>> Obtained best and overtrained for model 17
>>> Obtained best and overtrained for model 18
>>> Obtained best and overtrained for model 19
>>> Obtained best and overtrained for model 20
>>> Obtained best and overtrained for model 21
>>> Obtained best and 

40000/40000 [==============================] - 11s 283us/step - loss: 0.1206 - acc: 0.9691 - val_loss: 0.8605 - val_acc: 0.8230
Epoch 12/12
40000/40000 [==============================] - 11s 283us/step - loss: 0.0962 - acc: 0.9767 - val_loss: 0.8642 - val_acc: 0.8344
>>> Obtained best and overtrained for model 80
>>> Begin train for model 81
Train on 40000 samples, validate on 10000 samples
Epoch 1/6
 - 11s - loss: 1.6037 - acc: 0.6112 - val_loss: 0.9763 - val_acc: 0.6745

Epoch 00001: val_loss improved from inf to 0.97626, saving model to ./exp-models/model-81-epoch-01.hdf5
Epoch 2/6
 - 11s - loss: 0.7808 - acc: 0.7381 - val_loss: 0.8786 - val_acc: 0.7168

Epoch 00002: val_loss improved from 0.97626 to 0.87865, saving model to ./exp-models/model-81-epoch-02.hdf5
Epoch 3/6
 - 11s - loss: 0.6167 - acc: 0.7935 - val_loss: 0.8383 - val_acc: 0.7410

Epoch 00003: val_loss improved from 0.87865 to 0.83831, saving model to ./exp-models/model-81-epoch-03.hdf5
Epoch 4/6
 - 11s - loss: 0.5011 - 

Epoch 8/15
40000/40000 [==============================] - 11s 281us/step - loss: 0.2406 - acc: 0.9268 - val_loss: 0.7725 - val_acc: 0.8161
Epoch 9/15
40000/40000 [==============================] - 11s 282us/step - loss: 0.1959 - acc: 0.9421 - val_loss: 0.7848 - val_acc: 0.8207
Epoch 10/15
40000/40000 [==============================] - 11s 281us/step - loss: 0.1606 - acc: 0.9550 - val_loss: 0.7894 - val_acc: 0.8309
Epoch 11/15
40000/40000 [==============================] - 11s 281us/step - loss: 0.1306 - acc: 0.9651 - val_loss: 0.8217 - val_acc: 0.8319
Epoch 12/15
40000/40000 [==============================] - 11s 282us/step - loss: 0.1015 - acc: 0.9756 - val_loss: 0.8484 - val_acc: 0.8371
Epoch 13/15
40000/40000 [==============================] - 11s 281us/step - loss: 0.0833 - acc: 0.9817 - val_loss: 0.8468 - val_acc: 0.8440
Epoch 14/15
40000/40000 [==============================] - 11s 281us/step - loss: 0.0656 - acc: 0.9871 - val_loss: 0.8647 - val_acc: 0.8448
Epoch 15/15
40000/4000

Epoch 14/15
40000/40000 [==============================] - 11s 279us/step - loss: 0.0606 - acc: 0.9880 - val_loss: 0.9351 - val_acc: 0.8403
Epoch 15/15
40000/40000 [==============================] - 11s 280us/step - loss: 0.0469 - acc: 0.9927 - val_loss: 0.9455 - val_acc: 0.8438
>>> Obtained best and overtrained for model 87
>>> Begin train for model 88
Train on 40000 samples, validate on 10000 samples
Epoch 1/6
 - 11s - loss: 1.6002 - acc: 0.6008 - val_loss: 0.9908 - val_acc: 0.6705

Epoch 00001: val_loss improved from inf to 0.99079, saving model to ./exp-models/model-88-epoch-01.hdf5
Epoch 2/6
 - 11s - loss: 0.7963 - acc: 0.7315 - val_loss: 0.8948 - val_acc: 0.7164

Epoch 00002: val_loss improved from 0.99079 to 0.89479, saving model to ./exp-models/model-88-epoch-02.hdf5
Epoch 3/6
 - 11s - loss: 0.6367 - acc: 0.7864 - val_loss: 0.8196 - val_acc: 0.7476

Epoch 00003: val_loss improved from 0.89479 to 0.81961, saving model to ./exp-models/model-88-epoch-03.hdf5
Epoch 4/6
 - 11s - los

 - 11s - loss: 0.6215 - acc: 0.7884 - val_loss: 0.8270 - val_acc: 0.7363

Epoch 00003: val_loss improved from 0.89240 to 0.82697, saving model to ./exp-models/model-91-epoch-03.hdf5
Epoch 4/6
 - 11s - loss: 0.5058 - acc: 0.8297 - val_loss: 0.8023 - val_acc: 0.7561

Epoch 00004: val_loss improved from 0.82697 to 0.80233, saving model to ./exp-models/model-91-epoch-04.hdf5
Epoch 5/6
 - 11s - loss: 0.4133 - acc: 0.8620 - val_loss: 0.7849 - val_acc: 0.7758

Epoch 00005: val_loss improved from 0.80233 to 0.78495, saving model to ./exp-models/model-91-epoch-05.hdf5
Epoch 6/6
 - 11s - loss: 0.3390 - acc: 0.8889 - val_loss: 0.7876 - val_acc: 0.7869

Epoch 00006: val_loss did not improve from 0.78495
>>> Best epoch found: 5 Overtraining to: 12
Train on 40000 samples, validate on 10000 samples
Epoch 7/12
40000/40000 [==============================] - 11s 280us/step - loss: 0.2765 - acc: 0.9124 - val_loss: 0.7925 - val_acc: 0.7984
Epoch 8/12
40000/40000 [==============================] - 11s 279u

40000/40000 [==============================] - 11s 279us/step - loss: 0.1457 - acc: 0.9597 - val_loss: 0.8213 - val_acc: 0.8237
Epoch 11/15
40000/40000 [==============================] - 11s 279us/step - loss: 0.1176 - acc: 0.9702 - val_loss: 0.8407 - val_acc: 0.8280
Epoch 12/15
40000/40000 [==============================] - 11s 280us/step - loss: 0.0921 - acc: 0.9789 - val_loss: 0.8537 - val_acc: 0.8309
Epoch 13/15
40000/40000 [==============================] - 11s 279us/step - loss: 0.0738 - acc: 0.9840 - val_loss: 0.8743 - val_acc: 0.8320
Epoch 14/15
40000/40000 [==============================] - 11s 279us/step - loss: 0.0595 - acc: 0.9883 - val_loss: 0.8933 - val_acc: 0.8390
Epoch 15/15
40000/40000 [==============================] - 11s 280us/step - loss: 0.0453 - acc: 0.9927 - val_loss: 0.9086 - val_acc: 0.8402
>>> Obtained best and overtrained for model 94
>>> Begin train for model 95
Train on 40000 samples, validate on 10000 samples
Epoch 1/6
 - 11s - loss: 1.5911 - acc: 0.6093 

Epoch 13/15
40000/40000 [==============================] - 11s 279us/step - loss: 0.0786 - acc: 0.9833 - val_loss: 0.9518 - val_acc: 0.8281
Epoch 14/15
40000/40000 [==============================] - 11s 280us/step - loss: 0.0624 - acc: 0.9882 - val_loss: 0.9855 - val_acc: 0.8248
Epoch 15/15
40000/40000 [==============================] - 11s 281us/step - loss: 0.0511 - acc: 0.9916 - val_loss: 1.0095 - val_acc: 0.8275
>>> Obtained best and overtrained for model 97
>>> Begin train for model 98
Train on 40000 samples, validate on 10000 samples
Epoch 1/6
 - 11s - loss: 1.5794 - acc: 0.6050 - val_loss: 0.9772 - val_acc: 0.6769

Epoch 00001: val_loss improved from inf to 0.97725, saving model to ./exp-models/model-98-epoch-01.hdf5
Epoch 2/6
 - 11s - loss: 0.7842 - acc: 0.7350 - val_loss: 0.8719 - val_acc: 0.7170

Epoch 00002: val_loss improved from 0.97725 to 0.87191, saving model to ./exp-models/model-98-epoch-02.hdf5
Epoch 3/6
 - 11s - loss: 0.6256 - acc: 0.7899 - val_loss: 0.8146 - val_acc